# Planet Features API Python Client Introduction


This tutorial is an introduction to [Planet's](https://www.planet.com) Features API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `Planet` module. The Features API is used to create *Collections* made up of *Features* that follow [OGC-compliant features standards](https://ogcapi.ogc.org/features/). The features are stored as GeoJson format FeatureCollections, and will be then be available to use across the Planet APIs or platform, including in Planet Explorer and Features Manager. Your features  are referred to as *items* within the FeatureCollection by the API and API documentation.

## Requirements

An account on the [Planet Platform](https://www.planet.com/account/) is required to access any of Planet's API's. If you are not logged in, you will be prompted to do so below in the *SDK Authentication* section.

## Useful links 
* [Planet SDK for Python](https://planet-sdk-for-python.readthedocs.io/en/stable/get-started/quick-start-guide/)
* [Planet Python Client Repo](https://github.com/planetlabs/planet-client-python)
* [Planet Features API Documentation](https://docs.planet.com/develop/apis/features/)

The basic workflow for interaction with the Features API is:
1. Define Feature(s) through filepath or text.
2. Create a Collection with a unique name.
3. Add features to the collection.
4. Use feature reference code 'pl:ref' as geometry input in other API calls.

Collections and the Features they hold will be viewable in the Planet Platform [Features Manager](https://www.planet.com/features/). You should be able to see all the Features your org has access to.
<br><br>

Using the workflow in this tutorial will walk you through using a predefined AOI in Cairo, Egypt around Cairo Lake as the feature for our Collection. You may also input your own Feature Collection from an area of your choice.

____

## Set up

In order to interact with the Planet API using the Python client, we need to import the necessary packages and authenticate our Planet account credentials.

### Imports

In [68]:
import os
import geopandas as gpd
import planet
from planet import Auth, Planet, Session, order_request

### SDK Authentication

Your Planet login is used to authenticate and activate the Python SDK. You will be prompted via a link below to login and confirm on the page that the code displayed matches the authorization code printed. If this is your first time accessing the Planet SDK, you will also be prompted first to authorize the SDK access to your Planet account. 
If you would like to know more, please visit the [authentication documentation](https://docs.planet.com/develop/authentication).

In [ ]:
# OAuth2 python client authentication
# If you are not already logged in, this will prompt you to open a web browser to log in.

auth = Auth.from_profile('planet-user', save_state_to_storage=False)
auth.ensure_initialized(allow_open_browser=True, allow_tty_prompt=True)

session = Session(auth)
pl = Planet(session)

_____

### Define Features

First, define the FeatureCollection you wish to post to the FeaturesAPI. 

The features must follow the same rules for AOI geometry used in other Planet APIs, including:

- Standard GeoJSON FeatureCollection format.
- Must use WGS84/EPSG:4326 projection.
- Be 2 dimensional.
- Contain 1500 vertices or *less* per feature.

Full feature rules may be found [here](https://docs.planet.com/develop/apis/features/uploading-and-validating-features/#rules-for-creating-a-feature). 
<br> <br>
Below we have defined a FeatureCollection of two areas in Cairo, Egypt. You may use this FeatureCollection to follow along with the tutorial, define your own, or import one from a filepath of something like a geojson.

You have a few options to define features to post to your Collections:
- The most simple option is to use a predefined FeatureCollection, so that the Features API can separate each feature into individual items for you.
- You may alternatively pass in just the geometries from feature collections. 
    - There is a free tool [geojson.io](https://geojson.io/) that will allow you to create geoetries and FeatureCollections by hand. You can also draw and download custom AOI geojson's from [Planet Explorer](https://www.planet.com/explorer/).

Features can have properties and keys beyond just the `geometry` and the `id`. If a feature is uploaded without specifying the `property_id`, all properties of the feature are maintained in the item.

You can manually have your Feature's `id` be set with an 'id' key. You may also give you feature a display name in the Feature Manager web interface with a 'title' key in the features property. **Note** that when you upload a feature, it is not required to have a title. Your feature will still display without a 'Feature Name' in the [Features Manager](https://www.planet.com/features/) GUI, and is usable and accessible via its `id`. 


In [4]:
lake_poly = [[[31.23905453, 29.99579775], [31.26630889, 29.99579775], [31.26630889, 30.01908084], [31.23905453, 30.01908084], [31.23905453, 29.99579775]]]
institute_poly = [[[31.23344423, 30.05109414], [31.25491533, 30.05109414], [31.25491533, 30.06461941], [31.23344423, 30.06461941], [31.23344423, 30.05109414]]]

cairo_fc = {
    "type": "FeatureCollection",
    "features": [
        {
            "id": "0",
            "type": "Feature",
            "properties": {
                "title": "lake_area",
                "name_str": "area_1",
                "extra_property_1": 'extra property 1 here',
                "extra_property_2": 'extra property 2 here'},
            "geometry": {"coordinates": lake_poly, "type": "Polygon"},
        },
        {
            "id": "1",
            "type": "Feature",
            "properties": {
                "title": "institute_area",
                "name_str": "area_2",
                "extra_property_1": 'extra property 1 here',
                "extra_property_2": 'extra property 2 here'},
            "geometry": {"coordinates": institute_poly, "type": "Polygon"},
        },
]}

In [ ]:
# Read your geojson path, then send it to a geo_dict.
geojson_file = 'PATH TO YOUR GEOJSON FILE'

gdf = gpd.read_file(geojson_file)

local_fc = gdf.to_geo_dict()

___

### Name and create your FeatureCollection in the Features API

Before you upload your FeatureCollection items, you need to create the Collection with the Features API. Set your `collection_title`, which will serve as the title or name of the whole FeatureCollection once its uploaded. You may optionally set a description for your Collection. Then we can create our new Collection using `pl.features.create_collection`, which returns the newly created Collections's `collection_id`. The `collection_id` is needed to add items to the Collection.

In [6]:
collection_title = 'cairo_areas'
description = 'FeatureCollection of two areas in Cario, Egypt'

In [7]:
new_collection_id = pl.features.create_collection(title=collection_title, description=description)

In [8]:
new_collection_id

'cairo_areas-orELyjq'

### Post Features to your Collection

After the Collection is created with the Features API, you may post your features from the FeatureCollection we defined above. 
<br> <br>
The `property_id` is optional, it refers to which key from the `properties` block of the supplied feature(s) to use in the `id` for the collection. This gives you some control of the identifier for items once they are in your collection. Note that an alphanumeric string will be added onto the end of the id even if you set the key to use.
<br> <br>
Without the `property_id` specified, if the features has an `id` key, that will be used as the `id` when you post your feature. Otherwise, a random string is generated for a features' id. 
<br> <br>
The example FeatureColleciton has some extra properties like `name_str` that can be used as the id, but will retain all keys if posted without a `property_id` key.

In [9]:
# Optional
property_id = None

In [10]:
new_items = pl.features.add_items(collection_id=new_collection_id, feature = cairo_fc, property_id=property_id)

In [11]:
new_items

['pl:features/my/cairo_areas-orELyjq/0-m2z6LNQ',
 'pl:features/my/cairo_areas-orELyjq/1-o6DvKBY']

`add_items` returned a list of strings, which are the `pl reference ids` of the items we just added to the colleciton. These string are what we can use to as the `geometry` input for Orders and Subscriptions later. Before we order imagery with these, we will first explore the other functionalities of the Features python client.

____

### Access your feature Collecitons from the Features API

Basic information (title, description, extent, etc.) about all the feature collections you have access to is available with `list_collections()`

In [ ]:
collections = pl.features.list_collections()
collections_list = [collection for collection in collections]
collections_list

[[{'id': 'cairo_areas-orELyjq',
   'title': 'cairo_areas',
   'description': 'FeatureCollection of two areas in Cario, Egypt',
   'item_type': 'feature',
   'created': '2025-12-12T18:27:21Z',
   'updated': '2025-12-12T18:34:58Z',
   'extent': {'spatial': {'bbox': [[31.23344423,
       29.99579775,
       31.26630889,
       30.06461941]]}},
   'links': [{'href': 'https://api.planet.com/features/v1/ogc/my/collections/cairo_areas-orELyjq',
     'rel': 'self',
     'title': 'This collection',
     'type': 'application/json'},
    {'href': 'https://api.planet.com/features/v1/ogc/my/collections/cairo_areas-orELyjq/items',
     'rel': 'features',
     'title': 'Features',
     'type': 'application/json'}],
   'feature_count': 2,
   'area': 9890943.929543883,
   'title_property': None,
   'description_property': None,
   'properties': {},
   'permissions': {'can_write': True, 'shared': False, 'is_owner': True},
   'ownership': {'owner_id': '<redacted>', 'org_id': '<redacted>'},
   'pl:ref': '

Individual Collection items can be listed with `list_items()`. You can then use the item ids' returned from the items to access the individual items.

We will use the collection_id of the collection we created above, `new_collection_id`:

In [13]:
items = pl.features.list_items(new_collection_id)
item_list = [item for item in items]
item_list

[{'type': 'Feature',
  'id': '1',
  'properties': {'title': 'institute_area',
   'name_str': 'area_2',
   'extra_property_1': 'extra property 1 here',
   'extra_property_2': 'extra property 2 here',
   'pl:ref': 'pl:features/my/cairo_areas-orELyjq/1-o6DvKBY',
   'pl:area': 3104292.7715780144},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[31.23344423, 30.05109414],
     [31.25491533, 30.05109414],
     [31.25491533, 30.06461941],
     [31.23344423, 30.06461941],
     [31.23344423, 30.05109414]]]}},
 {'type': 'Feature',
  'id': '0',
  'properties': {'title': 'lake_area',
   'name_str': 'area_1',
   'extra_property_1': 'extra property 1 here',
   'extra_property_2': 'extra property 2 here',
   'pl:ref': 'pl:features/my/cairo_areas-orELyjq/0-m2z6LNQ',
   'pl:area': 6786651.157965869},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[31.23905453, 29.99579775],
     [31.26630889, 29.99579775],
     [31.26630889, 30.01908084],
     [31.23905453, 30.01908084],
     [31.23905453

To get an individual item from the feature collection, you need the `collection_id` as well as the `feature_id` itself. We will be reusing the `collection_id` from before, and using the first item in our list in that collection.

In [14]:
collection_id = new_collection_id
feature_id = item_list[0]['id']

In [15]:
pl.features.get_item(collection_id, feature_id)

{'type': 'Feature',
 'id': '1',
 'properties': {'title': 'institute_area',
  'name_str': 'area_2',
  'extra_property_1': 'extra property 1 here',
  'extra_property_2': 'extra property 2 here',
  'pl:ref': 'pl:features/my/cairo_areas-orELyjq/1-o6DvKBY',
  'pl:area': 3104292.7715780144},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[31.23344423, 30.05109414],
    [31.25491533, 30.05109414],
    [31.25491533, 30.06461941],
    [31.23344423, 30.06461941],
    [31.23344423, 30.05109414]]]},
 'links': [{'rel': 'self',
   'href': 'https://api.planet.com/features/v1/ogc/my/collections/cairo_areas-orELyjq/items/1-o6DvKBY'},
  {'rel': 'collection',
   'href': 'https://api.planet.com/features/v1/ogc/my/collections/cairo_areas-orELyjq'}]}

___

### Deleting Collections

Using the `collection_id` you may delete a collection using `delete_collection()`. Be careful when you do this, as there is no confirmation output from this method, and the collection will just be deleted.

In [ ]:
coll_id = ''
pl.features.delete_collection(coll_id)

_____

# Using Feature References as input geometries

The `'pl:ref'` property of a feature is used in other Planet APIs to link to the geometry of the feature instead of having the user track bounding box and geometry variables. Planet `Subscriptions`, `Orders`, and `Data` APIs are all able to make use of these pl feature reference links. 

PL ref links are submitted in the `geometry` argument in API calls as a dictionary: 
```python
geometry = {
    'type': 'ref',
    'content': 'pl:features/my/<collection_id>/<item_id>'
}
```

Provided below is consolidated code to use the SDK to get a `pl:ref` id from the features API, and then input that id into an order request. If you are unfamiliar with the mechanics of the `order_request` functions of the SDK, there is a full notebook, `planet_sdk_orders_demo.ipynb` located in `api_guids/orders_api/` that will walk you through how to use the Planet Python SDK to create and submit Orders.

In [29]:
### SDK Order workflow
## example: get the first Collection you have, and use the id of its first item as a geometry in an Order

# list all your collections
collections = pl.features.list_collections()
collections_list = [collection for collection in collections]

# Get the id of the first collection in your list
collection_id = collections_list[0]['id']

collection_items = list(pl.features.list_items(collection_id))
feature_ref = collection_items[1]['properties']['pl:ref']
geometry = {'type': 'ref', 'content': feature_ref}
geometry

{'type': 'ref', 'content': 'pl:features/my/cairo_areas-orELyjq/0-m2z6LNQ'}

In [ ]:
# This example item_id can be used to order using the example FeatureCollection given.
# You will get an error if you tried to order this item with a different feature.
item_ids = ['20220501_074038_04_2420']
product_bundle = 'analytic_8b_udm2'
item_type = "PSScene"
fallback_bundle = 'analytic_udm2,analytic_3b_udm2'

prod = order_request.product(item_ids, product_bundle, item_type, fallback_bundle)
tools = [order_request.clip_tool(geometry)]

In [64]:
name = 'geometry_ref_order'
products = [prod]

order_made = planet.order_request.build_request(name, products, tools=tools)

In [65]:
order_made

{'name': 'geometry_ref_order',
 'products': [{'item_ids': ['20220501_074038_04_2420'],
   'item_type': 'PSScene',
   'product_bundle': 'analytic_8b_udm2,analytic_udm2,analytic_3b_udm2'}],
 'tools': [{'clip': {'aoi': {'type': 'ref',
     'content': 'pl:features/my/cairo_areas-orELyjq/0-m2z6LNQ'}}}]}

In [66]:
order = pl.orders.create_order(request=order_made)

In [67]:
order

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/8d83f77c-e6f2-4699-9662-eb4a10b47647'},
 'created_on': '2025-12-12T19:20:44.501167Z',
 'error_hints': [],
 'id': '8d83f77c-e6f2-4699-9662-eb4a10b47647',
 'last_message': 'Preparing order',
 'last_modified': '2025-12-12T19:20:44.501167Z',
 'name': 'geometry_ref_order',
 'products': [{'item_ids': ['20220501_074038_04_2420'],
   'item_type': 'PSScene',
   'product_bundle': 'analytic_8b_udm2'}],
 'state': 'queued',
 'tools': [{'clip': {'aoi': {'content': 'pl:features/my/cairo_areas-orELyjq/0-m2z6LNQ',
     'type': 'ref'}}}]}